In [14]:
%pip install polars[all] pandas geopandas plotly folium

Note: you may need to restart the kernel to use updated packages.


In [15]:
import polars as pl
import geopandas as gpd
import pandas as pd
import folium

In [16]:
geojson_filepath = "./vendor/geo-countries/data/countries.geojson"
country_codes_df = pl.read_csv('data/country_codes.csv')
df_uchazeci = pl.read_csv("data/prf_uchazeci.csv").join(other=country_codes_df, left_on="STAT_OBCANSTVI", right_on="country_name", how="left")

# import geojson
# with open(geojson_filepath) as f:
#     gj = geojson.load(f)
# features = gj['features'][0]
# features["properties"]

In [17]:
gdf = gpd.read_file(geojson_filepath)

In [18]:

# Filter the GeoDataFrame based on the ISO_A3 value
selected_feature = gdf[gdf['ISO_A3'] == "ABW"]

m = folium.Map(location=[selected_feature['geometry'].centroid.y.mean(), selected_feature['geometry'].centroid.x.mean()], zoom_start=4)

# Add the selected feature to the map
folium.GeoJson(selected_feature).add_to(m)

# Add a text label with a value to the centroid of the selected feature
label_value = 100  # Replace with your specific value
folium.Marker(
    location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],
    icon=folium.DivIcon(html=f'<div style="font-size: 16pt; color: red;">{label_value}</div>')
).add_to(m)

m




/tmp/ipykernel_2786810/3663858707.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[selected_feature['geometry'].centroid.y.mean(), selected_feature['geometry'].centroid.x.mean()], zoom_start=4)
/tmp/ipykernel_2786810/3663858707.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],


In [19]:
stat = df_uchazeci.select("STAT_OBCANSTVI")
joined_df = stat.join(other=country_codes_df, left_on="STAT_OBCANSTVI", right_on="country_name", how="left")
joined_df


STAT_OBCANSTVI,official_name_en,ISO3166-1-Alpha-3
str,str,str
"""Česká republik…","""Czechia""","""CZE"""
"""Česká republik…","""Czechia""","""CZE"""
"""Česká republik…","""Czechia""","""CZE"""
"""Česká republik…","""Czechia""","""CZE"""
"""Česká republik…","""Czechia""","""CZE"""
…,…,…
"""Česká republik…","""Czechia""","""CZE"""
"""Česká republik…","""Czechia""","""CZE"""
"""Česká republik…","""Czechia""","""CZE"""


In [20]:
joined_df = joined_df.group_by("ISO3166-1-Alpha-3").agg(pl.count("STAT_OBCANSTVI").alias("count"))

In [21]:
joined_df

ISO3166-1-Alpha-3,count
str,u32
"""CZE""",4441
"""KGZ""",5
"""ARM""",2
"""IRN""",3
"""LBN""",1
…,…
"""BLR""",10
null,2
"""IDN""",1


In [22]:
# selected_feature = gdf[gdf['ISO_A3'] == "CZE"]

# m = folium.Map(location=[selected_feature['geometry'].centroid.y.mean(), selected_feature['geometry'].centroid.x.mean()], zoom_start=4)
m = folium.Map()

# Add the selected feature to the map
# folium.GeoJson(selected_feature).add_to(m)

# Add a text label with a value to the centroid of the selected feature
for row in joined_df.iter_rows(named=True):
    obec = row["ISO3166-1-Alpha-3"]
    print(obec)
    if obec is None:
        continue
    count = row["count"]
    selected_feature = gdf[gdf['ISO_A3'] == obec]
    folium.GeoJson(selected_feature).add_to(m)
    # folium.GeoJson(selected_feature).add_to(m)
    # label_value = row[1]

    # if len(selected_feature['geometry'].centroid.y.values) == 0:
    #     continue
    # if len(selected_feature['geometry'].centroid.x.values) == 0:
    #     continue

    folium.Marker(
        location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],
        icon=folium.DivIcon(html=f'<div style="font-size: 11pt; color: black;">{obec}:{count}</div>')
    ).add_to(m)
m

/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],
/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],
/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].

CZE
KGZ
ARM
IRN
LBN
HRV
MDA
KAZ
SVK
ISR
YEM
NAM
UZB


/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],
/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],
/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].

AZE
HUN
RUS
VNM


/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],
/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],
/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].

DEU
AFG
BGR
EGY
KOR
BLR
None
IDN
UKR


/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],
/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].centroid.x.values[0]],
/tmp/ipykernel_2786810/2156574706.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[selected_feature['geometry'].centroid.y.values[0], selected_feature['geometry'].

PER


In [23]:
import polars as pl
import geopandas as gpd
import pandas as pd
import folium

In [24]:
df = pl.read_csv("data/prf_uchazeci.csv").join(other=country_codes_df, left_on="STAT_OBCANSTVI", right_on="country_name", how="left")
df = df.filter(pl.col("ISO3166-1-Alpha-3") == "CZE")
df.shape


(4441, 31)

In [25]:
gdf = gpd.read_file("./vendor/obce-transformed.geojson")
# NOTE: PSC has 0s
# df = pl.read_csv("data/prf_uchazeci.csv").filter(pl.col("OBEC_BYDLISTE") != "nevyplněno", pl.col("OBEC_BYDLISTE") != "Osek")
df = pl.read_csv("data/prf_uchazeci.csv").filter(pl.col("OBEC_BYDLISTE") != "nevyplněno")
places_df = pl.read_csv("data/places.csv")
df

FAKULTA_SP,ROK_NAR,STAT_OBCANSTVI,PSC_BYDL,PSC_CIZI,OBEC_BYDLISTE,OBEC_CIZI,STAT_BYDLISTE,NAZEV_S_SKOLY,STAT_S_SKOLY,ADRESA_S_SKOLY,CIS_OBORU,MATURITA,STRED_SKOLA_CIZI,STRED_SKOLA_CIZI_MISTO,STRED_SKOLA_CIZI_OBOR,VS_SKOLA,NAZEV_VS,STAT_VS,VS_ROK_ABSOL,ROK_PR,CISP_KOD,CZ_NAZEV,FORMA,TYP,PR_NAZEV,PR_OBOR,PR_KOMBINACE,SDRUZENE
str,i64,str,i64,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,f64,i64,str,str,str,str,str,str,str,str
"""PRF""",1998,"""Česká republik…",41705,null,"""Osek""",null,"""Česká republik…","""Gymnázium T. G…","""Česká republik…","""Litvínov, Stud…","""7941K41""",2018,null,null,null,null,null,null,null,2019,"""B1101""","""Matematika""","""P""","""B""","""MA BI1""",null,"""MA BI""",null
"""PRF""",2000,"""Česká republik…",40001,null,"""Ústí nad Labem…",null,"""Česká republik…","""Gymnázium""","""Česká republik…","""Ústí nad Labem…","""7941K41""",2019,null,null,null,"""Univerzita Jan…","""Univerzita Jan…","""Česká republik…",2023.0,2019,"""B1101""","""Matematika""","""P""","""B""","""MA GEO1""",null,"""MA GEO""",null
"""PRF""",2000,"""Česká republik…",41752,null,"""Hostomice""",null,"""Česká republik…","""VOŠ zdravotnic…","""Česká republik…","""Ústí nad Labem…","""7842M04""",2019,null,null,null,"""Univerzita Jan…","""Univerzita Jan…","""Česká republik…",2023.0,2019,"""B1101""","""Matematika""","""P""","""B""","""MA GEO2""",null,"""MA GEO""",null
"""PRF""",2000,"""Česká republik…",41742,null,"""Krupka""",null,"""Česká republik…","""Biskupské gymn…","""Česká republik…","""Krupka, Koněvo…","""7941K81""",2019,null,null,null,"""Univerzita Jan…","""Univerzita Jan…","""Česká republik…",2020.0,2019,"""B1101""","""Matematika""","""P""","""B""","""MA TV1""",null,"""MA TV""",null
"""PRF""",2000,"""Česká republik…",40001,null,"""Ústí nad Labem…",null,"""Česká republik…","""Gymnázium""","""Česká republik…","""Ústí nad Labem…","""7941K41""",2019,null,null,null,"""Univerzita Jan…","""Univerzita Jan…","""Česká republik…",2023.0,2019,"""B1101""","""Matematika""","""P""","""B""","""MA TV1""",null,"""MA TV""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""PRF""",1997,"""Česká republik…",27601,null,"""Mělník""",null,"""Česká republik…","""Gymnázium Jana…","""Česká republik…","""Mělník, Pod Vr…","""7941K41""",2017,null,null,null,"""Univerzita Jan…","""Univerzita Jan…","""Česká republik…",2023.0,2023,"""P0532D330032""","""Geographies of…","""P""","""D""","""D GEOTR2""",null,null,null
"""PRF""",1997,"""Česká republik…",41201,null,"""Litoměřice""",null,"""Česká republik…","""Gymnázium Jose…","""Česká republik…","""Litoměřice, Sv…","""7941K41""",2016,null,null,null,"""Univerzita Jan…","""Univerzita Jan…","""Česká republik…",2023.0,2023,"""P0533D110037""","""Počítačové mod…","""P""","""D""","""D PMVT2""",null,null,null
"""PRF""",1997,"""Česká republik…",43921,null,"""Koštice""",null,"""Česká republik…","""Gymnázium""","""Česká republik…","""Lovosice, Sady…","""7941K41""",2017,null,null,null,"""Univerzita Jan…","""Univerzita Jan…","""Česká republik…",2023.0,2023,"""P0533D110049""","""Aplikované ion…","""P""","""D""","""D AIT2""",null,null,null


In [26]:
places_df.head()

obec_id,obec_name,orp_id,orp_name,okres_id,okres_name,kraj_id,kraj_name
i64,str,i64,str,i64,str,i64,str
554979,"""Abertamy""",4106,"""Ostrov""",3403,"""Karlovy Vary""",51,"""Karlovarský kr…"
531367,"""Adamov""",2104,"""Čáslav""",3205,"""Kutná Hora""",27,"""Středočeský kr…"
535826,"""Adamov""",3102,"""České Budějovi…",3301,"""České Budějovi…",35,"""Jihočeský kraj…"
581291,"""Adamov""",6201,"""Blansko""",3701,"""Blansko""",116,"""Jihomoravský k…"
547786,"""Adršpach""",5201,"""Broumov""",3605,"""Náchod""",86,"""Královéhradeck…"


In [27]:
df = df.join(other=places_df, how="left", right_on="obec_name", left_on="OBEC_BYDLISTE")
df.head()

FAKULTA_SP,ROK_NAR,STAT_OBCANSTVI,PSC_BYDL,PSC_CIZI,OBEC_BYDLISTE,OBEC_CIZI,STAT_BYDLISTE,NAZEV_S_SKOLY,STAT_S_SKOLY,ADRESA_S_SKOLY,CIS_OBORU,MATURITA,STRED_SKOLA_CIZI,STRED_SKOLA_CIZI_MISTO,STRED_SKOLA_CIZI_OBOR,VS_SKOLA,NAZEV_VS,STAT_VS,VS_ROK_ABSOL,ROK_PR,CISP_KOD,CZ_NAZEV,FORMA,TYP,PR_NAZEV,PR_OBOR,PR_KOMBINACE,SDRUZENE,obec_id,orp_id,orp_name,okres_id,okres_name,kraj_id,kraj_name
str,i64,str,i64,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,f64,i64,str,str,str,str,str,str,str,str,i64,i64,str,i64,str,i64,str
"""PRF""",1998,"""Česká republik…",41705,null,"""Osek""",null,"""Česká republik…","""Gymnázium T. G…","""Česká republik…","""Litvínov, Stud…","""7941K41""",2018,null,null,null,null,null,null,null,2019,"""B1101""","""Matematika""","""P""","""B""","""MA BI1""",null,"""MA BI""",null,531634,2108,"""Hořovice""",3202,"""Beroun""",27,"""Středočeský kr…"
"""PRF""",1998,"""Česká republik…",41705,null,"""Osek""",null,"""Česká republik…","""Gymnázium T. G…","""Česká republik…","""Litvínov, Stud…","""7941K41""",2018,null,null,null,null,null,null,null,2019,"""B1101""","""Matematika""","""P""","""B""","""MA BI1""",null,"""MA BI""",null,551562,3111,"""Strakonice""",3307,"""Strakonice""",35,"""Jihočeský kraj…"
"""PRF""",1998,"""Česká republik…",41705,null,"""Osek""",null,"""Česká republik…","""Gymnázium T. G…","""Česká republik…","""Litvínov, Stud…","""7941K41""",2018,null,null,null,null,null,null,null,2019,"""B1101""","""Matematika""","""P""","""B""","""MA BI1""",null,"""MA BI""",null,560057,3211,"""Rokycany""",3408,"""Rokycany""",43,"""Plzeňský kraj"""
"""PRF""",1998,"""Česká republik…",41705,null,"""Osek""",null,"""Česká republik…","""Gymnázium T. G…","""Česká republik…","""Litvínov, Stud…","""7941K41""",2018,null,null,null,null,null,null,null,2019,"""B1101""","""Matematika""","""P""","""B""","""MA BI1""",null,"""MA BI""",null,567779,4213,"""Teplice""",3509,"""Teplice""",60,"""Ústecký kraj"""
"""PRF""",1998,"""Česká republik…",41705,null,"""Osek""",null,"""Česká republik…","""Gymnázium T. G…","""Česká republik…","""Litvínov, Stud…","""7941K41""",2018,null,null,null,null,null,null,null,2019,"""B1101""","""Matematika""","""P""","""B""","""MA BI1""",null,"""MA BI""",null,573264,5207,"""Jičín""",3604,"""Jičín""",86,"""Královéhradeck…"
